In [20]:
#Чтобы получить список 250 фильмов на сайте Кинопоиска, нам необходима библиотека Requests - она позволяет нам делать запросы (те самые requests) и получать данные из интернета.
import requests 

In [21]:
url = 'https://www.kinopoisk.ru/lists/top250/' 
#Чтобы получить данные мы должны применить к адресу метода библиотеки get. Чтобы узнать, какую именно информацию мы получили, можно применить опцию text... Но делать этого мы, конечно же, не будем (если честно, я попробовал выгрузить на гитхаб вакриант с кодом - он получился безумно большим).
top = requests.get(url)

In [3]:
#Код получился нечитаемым (поверьте мне). Справиться с этим поможет другая библиотека, Beautifulsoup. Эта библиотека для извлечения данных из файлов HTML и XML. Она работает с библиотеками для парсинга, чтобы дать естественные способы навигации, поиска и изменения дерева разбора.
from bs4 import BeautifulSoup

In [22]:
#Мы должны передать библиотеке ответ, полученный с помощью Requests. Создадим переменную soup, передадим ей top.text и укажем необходимый формат (насколько я понял, наиболее быстрый и удобный - lxml)
soup = BeautifulSoup(top.text, 'lxml') #Давайте и ее тоже не будем печатать

In [19]:
#Код упорядочен (честно пионерское), теги идут один за одним — теперь мы можем вынимать из кода то, что нам нужно. Для этого просмотрим и проанализируем код страницы "Топ 250". Анализ показывает, что карточки каждого из фильмов лежат в теге div. С помощью команды find (и get для получения ссылки на фильм) вытащим данные из первой карточки, а затем перепишем код так, чтобы он сработал на все фильмы со страницы.
#Получаем ссылку на фильм. Она лежит в классе href (для него то и пригодится get), однако не полностью, поэтому добавим вначале адрес главной страницы сайта. 
link = 'https://www.kinopoisk.ru/'+soup.find('div', class_='desktop-rating-selection-film-item').find('a', class_='selection-film-item-meta__link').get('href')
print(link)

https://www.kinopoisk.ru//film/24705/


In [6]:
#Вытаскиваем русское название фильма
russian_name = soup.find('div', class_='desktop-rating-selection-film-item').find('a', class_='selection-film-item-meta__link').find('p', class_='selection-film-item-meta__name').text
print(russian_name)

Зеленая миля


In [7]:
#Забираем оригинальное название (для нас еще более важно, что в этом же теге располагается год выхода фильма)
original_name = soup.find('div', class_='desktop-rating-selection-film-item').find('a', class_='selection-film-item-meta__link').find('p', class_='selection-film-item-meta__original-name').text
print(original_name)

Green Mile, The, 1999


In [8]:
#Ищем страну-производителя фильма
country = soup.find('div', class_='desktop-rating-selection-film-item').find('a', class_='selection-film-item-meta__link').find('span', class_='selection-film-item-meta__meta-additional-item').text
print(country)

США


In [9]:
#Жанр. В коде Кинопоиска страна и жанр имеют одинаковый класс selection-film-item-meta__meta-additional-item — а команда find находит только первое соответствие вашему запросу. Применим команду findAll, чтобы получить список.
genre = soup.find('div', class_='desktop-rating-selection-film-item').find('a', class_='selection-film-item-meta__link').findAll('span', class_='selection-film-item-meta__meta-additional-item')[1].text
print(genre)

фэнтези, драма


In [10]:
#Рейтинг фильма на Кинопоиске
rate = soup.find('div', class_='desktop-rating-selection-film-item').find('span', class_='rating__value rating__value_positive').text
 print(rate)

8.9


In [12]:
#Чтобы давать меньшую нагрузку на сервер, эксперты советуют импортировать из библиотеки time функцию sleep. Пытался запустить код без нее - получалось хуже.
from time import sleep

In [13]:
#Сделаем так, чтобы код работал для всех фильмов
data = [] #переменную data, в который будем добавлять все переменные, которые запрашиваем.

for p in range(1, 6): #Список топ-250 разбит на 5 страниц по 50 фильмов. Чтобы код прогрузил данные всех фильмов, создадим цикл, который будет последовательно просматривать страницы.
    print(p)

    url = f"https://www.kinopoisk.ru/lists/top250/?page={p}&tab=all" #В каждом цикле меняется номер страницы (в адресной строке она записаны как page=<2,3,4>)
    top = requests.get(url)
    sleep(7)
    soup = BeautifulSoup(top.text, 'lxml')

    films = soup.findAll('div', class_='desktop-rating-selection-film-item') #Создаем поиск всех фильмов с уже знакомой нам командой findAll.

    for film in films:
        link = 'https://www.kinopoisk.ru/'+film.find('a', class_='selection-film-item-meta__link').get('href')
        russian_name = film.find('a', class_='selection-film-item-meta__link').find('p', class_='selection-film-item-meta__name').text
        original_name = film.find('a', class_='selection-film-item-meta__link').find('p', class_='selection-film-item-meta__original-name').text
        country = film.find('a', class_='selection-film-item-meta__link').find('span', class_='selection-film-item-meta__meta-additional-item').text
        genre = film.find('a', class_='selection-film-item-meta__link').findAll('span', class_='selection-film-item-meta__meta-additional-item')[1].text
        rate = film.find('span', class_='rating__value rating__value_positive').text
        data.append([link, russian_name, original_name, country, genre, rate])

1
2
3
4
5


In [14]:
#Проверяем, все ли запрашиваемые данные загрузились... И усердно танцуем с бубном.
len(data)

250

In [15]:
#Pandas - библиотека Python для анализа данных. Она позволяет работать с данными в табличном виде. В Pandas можно изучить набор данных, почистить его, внести изменения, проанализировать и сделать выводы, построить графики и многое другое. 
import pandas as pd

In [17]:
#Сейчас библиотека нам нужна, чтобы создать и сохранить таблицу.
header = ['Ссылка', 'Русское название', 'Оригинальное название, год', 'Страна', 'Жанр', 'Рейтинг']

df = pd.DataFrame(data, columns=header)
df.to_csv('/Users/ok_computer/Downloads/kinopoisk_top250.csv', sep=';', encoding='utf8')
df

,Ссылка,Русское название,"Оригинальное название, год",Страна,Жанр,Рейтинг
0,https://www.kinopoisk.ru//film/435/,Зеленая миля,"Green Mile, The, 1999",США,"фэнтези, драма",8.9
1,https://www.kinopoisk.ru//film/326/,Побег из Шоушенка,"Shawshank Redemption, The, 1994",США,драма,8.9
2,https://www.kinopoisk.ru//film/3498/,Властелин колец: Возвращение короля,"Lord of the Rings: The Return of the King, The...","Новая Зеландия, США","фэнтези, приключения",8.8
3,https://www.kinopoisk.ru//film/312/,Властелин колец: Две крепости,"Lord of the Rings: The Two Towers, The, 2002","Новая Зеландия, США","фэнтези, приключения",8.8
4,https://www.kinopoisk.ru//film/328/,Властелин колец: Братство Кольца,"Lord of the Rings: The Fellowship of the Ring,...","Новая Зеландия, США","фэнтези, приключения",8.8
...,...,...,...,...,...,...
245,https://www.kinopoisk.ru//film/770973/,Вселенная Стивена Хокинга,"Theory of Everything, The, 2014","Великобритания, Япония","биография, мелодрама",8.0
246,https://www.kinopoisk.ru//film/634254/,Ип Ман 4,"Yip Man 4, 2019","Гонконг, Китай","боевик, биография",8.0
247,https://www.kinopoisk.ru//film/316/,Матрица: Революция,"Matrix Revolutions, The, 2003",США,"фантастика, боевик",8.0
248,https://www.kinopoisk.ru//film/493208/,Холодное сердце,"Frozen, 2013","США, Норвегия","мультфильм, мюзикл",8.0
